In [ ]:
import sys
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import numpy as np
import pandas as pd
import can
from IPython.display import clear_output, display
sys.path.append('../Functions/py_functions/') # This path is so that within each function file, you can import the other function files with relative paths
sys.path.append('../') # This path is so that we can import the functions folder from the root directory compared to where this file is
from Functions.py_functions.constants import *

In [ ]:
# load log file and plot the data
# each row of the csv has the name of the data and the data itself
file_name = "charger_data_3.csv"
# load the data row by row
print("Loading data from file: " + file_name)
data = []
with open(file_name, 'r') as f:
    # read each row as a list and put that list into the data list
    for row in f:
        data.append(row.split(','))
# collect the data with its timestamps
data_dict = {}
for i in range(int(len(data)/2)):
    name = data[2*i][0][:-5]
    if name[-8:] == "_Voltage":
        name = name[:-8]
    else:
        name = name[:-5]
        
    # create a array for the data
    times = np.array(data[2*i][1:-1], dtype=float)
    datas = np.array(data[2*i+1][1:-1], dtype=float)
    data_dict[name] = np.array([times, datas])
print("Data loaded successfully!")

In [ ]:
# plot the voltages and temperatures of the battery pack
# function to clean out all the bad values for temperature
def clean_temp(temps):
    return temps[:, (temps[1,:]<100) & (temps[1,:]>0)]

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)
# get all the voltages
voltage_names = [name for name in data_dict.keys() if 'Voltage' in name]
# print(voltage_names)
# plot all the voltages, bus only every nth point
nth = 50
for name in voltage_names:
    fig.add_trace(go.Scattergl(x=data_dict[name][0,::nth], y=data_dict[name][1,::nth], mode="lines", name=name, line=dict(width=1)), row=1, col=1)

# get all the temperatures
temp_names = [name for name in data_dict.keys() if 'Temp' in name]
# print(temp_names)
# plot all the temperatures, bus only every nth point
nth = 1
for name in temp_names:
    fig.add_trace(go.Scattergl(x=data_dict[name][0,::nth], y=clean_temp(data_dict[name])[1,::nth], mode="lines", name=name, line=dict(width=1)), row=2, col=1)

# make the x axis show time in hours and minutes from seconds
fig.update_xaxes(tickformat="%H:%M:%S")
fig.update_layout(title="Voltages", xaxis_title="Time", yaxis_title="Voltage", template="plotly_dark", height=800)
fig.show()